# Topic: Suicide and Economic Stress

### How do macroeconomic conditions shape mental health outcomes?

Economic stressors (unemployment, job insecurity) are strongly linked to suicide risk, but their effects are uneven across demographic groups, which matters for targeted intervention.

# Setting Up

### Import Dependencies

In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image


### Helper Functions

In [59]:
def gho_data_with_details(code):

    url=(
        "https://ghoapi.azureedge.net/api/DIMENSION/GHO/DimensionValues"
        f"?$filter=Code eq '{code}'"
    )

    r = requests.get(url, timeout=30)
    r.raise_for_status()

    data = r.json()["value"][0]
    print(f"Dataset {data['Code']}: {data['Title']}\n")

    url = (
        f"https://ghoapi.azureedge.net/api/{code}"
        "?$filter=SpatialDim eq 'SGP'"
    )

    r = requests.get(url, timeout=30)
    r.raise_for_status()

    data = r.json()["value"]

    print(f"{len(data)} data points")

    sample = data[0]
    
    print("Sample data with the following fields:")

    for idx, k in enumerate(sample):
        print(f"[{idx+1:02d}] {k}: {sample[k]}")

    return data

# Macro Plots

### Dataset 1: Age-standardized suicide rates (per 100 000 population) in Singapore

In [112]:
data = gho_data_with_details(code="MH_12")

required_fields = [
    "TimeDim",
    "Dim1",
    "NumericValue",
    "Low",
    "High"
]

df = pd.DataFrame(data=data)
df = df[required_fields]
df = df[df["Dim1"] == "SEX_BTSX"]
df = df.sort_values(["TimeDim", "Dim1"])
df = df.reset_index(drop=True)

df.head(10)

Dataset MH_12: Age-standardized suicide rates (per 100 000 population)

66 data points
Sample data with the following fields:
[01] Id: 6287110
[02] IndicatorCode: MH_12
[03] SpatialDimType: COUNTRY
[04] SpatialDim: SGP
[05] TimeDimType: YEAR
[06] ParentLocationCode: WPR
[07] ParentLocation: Western Pacific
[08] Dim1Type: SEX
[09] TimeDim: 2011
[10] Dim1: SEX_MLE
[11] Dim2Type: None
[12] Dim2: None
[13] Dim3Type: None
[14] Dim3: None
[15] DataSourceDimType: None
[16] DataSourceDim: None
[17] Value: 10.0 [9.7-12.3]
[18] NumericValue: 10.02744978
[19] Low: 9.716854624
[20] High: 12.33761355
[21] Comments: None
[22] Date: 2025-01-28T14:13:24.673+01:00
[23] TimeDimensionValue: 2011
[24] TimeDimensionBegin: 2011-01-01T00:00:00+01:00
[25] TimeDimensionEnd: 2011-12-31T00:00:00+01:00


,TimeDim,Dim1,NumericValue,Low,High
0,2000,SEX_BTSX,11.450116,10.292107,13.164404
1,2001,SEX_BTSX,10.697307,9.482015,12.186357
2,2002,SEX_BTSX,11.400524,10.223714,13.054683
3,2003,SEX_BTSX,11.184666,10.143269,13.024308
4,2004,SEX_BTSX,10.415716,9.703154,12.385481
5,2005,SEX_BTSX,9.950540,9.119037,11.644053
6,2006,SEX_BTSX,9.834240,9.298902,11.842765
7,2007,SEX_BTSX,9.383625,8.485815,10.894244
8,2008,SEX_BTSX,8.755629,8.168445,10.423706
9,2009,SEX_BTSX,9.072118,8.262633,10.545650


In [ ]:
fig = go.Figure()

# Uncertainty band
fig.add_trace(go.Scatter(
    x=df["TimeDim"],
    y=df["High"],
    mode="lines",
    line=dict(width=0),
    showlegend=False,
    hoverinfo="skip"
))

fig.add_trace(go.Scatter(
    x=df["TimeDim"],
    y=df["Low"],
    mode="lines",
    line=dict(width=0),
    fill="tonexty",
    name="Uncertainty (Low–High)"
))

# Main line
fig.add_trace(go.Scatter(
    x=df["TimeDim"],
    y=df["NumericValue"],
    mode="lines+markers",
    name="Population Average (Both Sexes)"
))

# Only show years with data points
years = sorted(df["TimeDim"].unique())
fig.update_xaxes(tickmode="array", tickvals=years)

fig.update_layout(
    title="Singapore: Age-standardised Suicide Rate (Both Sexes)",
    xaxis_title="Year",
    yaxis_title="Suicide rate per 100,000 population",
    template="simple_white",
    hovermode="x unified"
)

fig.show()
pio.write_image(fig, "fig.png", scale=2)
Image("fig.png")

# Micro Plots

### Dataset 2: Crude suicide rates (per 100 000 population)

In [114]:
data = gho_data_with_details(code="SDGSUICIDE")

required_fields = [
    "TimeDim",
    "Dim1",
    "Dim2",
    "NumericValue",
    "Low",
    "High"
]

valid_age_groups = [
    "AGEGROUP_YEARS10-19",
    "AGEGROUP_YEARS20-29",
    "AGEGROUP_YEARS30-39",
    "AGEGROUP_YEARS40-49",
    "AGEGROUP_YEARS50-59",
    "AGEGROUP_YEARS60-69",
    "AGEGROUP_YEARS70PLUS",
]

df = pd.DataFrame(data=data)
df = df[required_fields]
df = df[df["TimeDim"] == 2021]
df = df[df["Dim2"].isin(valid_age_groups)]
df = df[df["Dim1"] == "SEX_BTSX"]
df = df.sort_values(["TimeDim", "Dim1", "Dim2"])
df = df.reset_index(drop=True)

df.head(10)

Dataset SDGSUICIDE: Crude suicide rates (per 100 000 population)

99 data points
Sample data with the following fields:
[01] Id: 6199220
[02] IndicatorCode: SDGSUICIDE
[03] SpatialDimType: COUNTRY
[04] SpatialDim: SGP
[05] ParentLocationCode: WPR
[06] TimeDimType: YEAR
[07] ParentLocation: Western Pacific
[08] Dim1Type: SEX
[09] TimeDim: 2019
[10] Dim1: SEX_FMLE
[11] Dim2Type: AGEGROUP
[12] Dim2: AGEGROUP_YEARSALL
[13] Dim3Type: None
[14] Dim3: None
[15] DataSourceDimType: None
[16] DataSourceDim: None
[17] Value: 5.2 [4.6-6.2]
[18] NumericValue: 5.233113969
[19] Low: 4.636493873
[20] High: 6.186538169
[21] Comments: None
[22] Date: 2025-01-10T10:52:31.77+01:00
[23] TimeDimensionValue: 2019
[24] TimeDimensionBegin: 2019-01-01T00:00:00+01:00
[25] TimeDimensionEnd: 2019-12-31T00:00:00+01:00


,TimeDim,Dim1,Dim2,NumericValue,Low,High
0,2021,SEX_BTSX,AGEGROUP_YEARS10-19,9.743928,NaN,NaN
1,2021,SEX_BTSX,AGEGROUP_YEARS20-29,8.100318,NaN,NaN
2,2021,SEX_BTSX,AGEGROUP_YEARS30-39,4.993701,NaN,NaN
3,2021,SEX_BTSX,AGEGROUP_YEARS40-49,6.570904,NaN,NaN
4,2021,SEX_BTSX,AGEGROUP_YEARS50-59,11.741348,NaN,NaN
5,2021,SEX_BTSX,AGEGROUP_YEARS60-69,12.089750,NaN,NaN
6,2021,SEX_BTSX,AGEGROUP_YEARS70PLUS,13.124246,NaN,NaN


In [ ]:
df["AgeGroupLabel"] = (
    df["Dim2"]
    .str.replace("AGEGROUP_YEARS", "")
    .str.replace("PLUS", "+")
)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=df["AgeGroupLabel"],
        y=df["NumericValue"],
        name="Both sexes",
        hovertemplate=
            "Age group: %{x}<br>" +
            "Suicide rate: %{y:.2f} per 100,000"
    )
)

fig.update_layout(
    title="Singapore: Suicide Rates by Age Group (2021, Both Sexes)",
    xaxis_title="Age group",
    yaxis_title="Suicide rate per 100,000 population",
    template="simple_white",
    hovermode="x"
)

fig.show()
pio.write_image(fig, "fig.png", scale=2)
Image("fig.png")